In [1]:
import tika
import os
# os.environ['TIKA_SERVER_JAR'] = "/var/folders/l6/2rm09dyd0d58tk10yvjy26f80000gn/T/tika-server.jar"
# os.environ['TIKA_SERVER_JAR'] = "/Users/chenchen/Desktop/tika-server-1.19.jar"


In [2]:
%%bash
cp /Users/chenchen/Desktop/tika-server-1.19.jar /var/folders/l6/2rm09dyd0d58tk10yvjy26f80000gn/T/

In [3]:
# get the doi
def get_doi(parsed_content):
    import re
    doi_pattern = re.compile('doi:?\s*([a-z0-9.\/]+)', re.I)
    if doi_pattern.search(parsed_content):
        doi = doi_pattern.search(parsed_content).group(1)
        return doi
    else:
        return None


In [4]:
# get the json from cross
def cross_doi(doi):
    import requests
    url_doi = "https://api.crossref.org/works/"
    search_url = url_doi + doi
    response = requests.get(search_url)
    print(response.json()['message']['title'][0])

In [5]:
# loop through this folder
import os
from tika import parser
for file in os.listdir('.'):
    if file.lower().endswith('.pdf'):
        parsed = parser.from_file(file)
        doi = get_doi(parsed['content'])
        if not doi:
            print("File {} have no doi".format(file))
#             print(parsed['content'])
        else:
            print("File {} have the doi: {}".format(file, doi))
            cross_doi(doi)

File lin2018.pdf have the doi: 10.1097/PRS.0000000000004849
Simultaneous Reconstruction of the Lower Lip with Gracilis Functioning Free Muscle Transplantation for Facial Reanimation
File zelken2016.pdf have the doi: 10.1097/SAP.0000000000000759
Silicone-Polytetrafluoroethylene Composite Implants for Asian Rhinoplasty
File li2016.pdf have the doi: 10.1097/SAP.0000000000000841
An Overview of Asian Rhinoplasty
File 10.1097@PRS.0000000000004948.pdf have the doi: 10.1097/PRS.0000000000004948
Anatomy of the Superficial Fascia System of the Breast
File 10.1097@PRS.0000000000004961.pdf have the doi: 10.1097/PRS.0000000000004961
Getting the Most Out of Augmentation-Mastopexy
File jabbour2018.pdf have the doi: 10.1097/PRS.0000000000004836
The Impact of Upper Face Botulinum Toxin Injections on Eyebrow Height and Forehead Lines
File stucker1999.pdf have no doi


# possible no doi
1. The paper is old
2. The paper is too new 
3. The paper is not published yet


In [33]:
# get the first lines of 
from tika import parser
from Bio import Entrez, Medline
# define the email to prevent block
Entrez.email = 'chenfmmu@163.com'

# get the first n lines excluding the trailing line breaks
def getfirstlines(pdffile, n=3):
    from tika import parser
    splitter = "\n\n"
    parsed = parser.from_file(file)
    content = parsed['content'].strip().split(splitter)

    return content[:n]

searchlist = getfirstlines('stucker1999.pdf')

# make the term and search in the title
TERM = searchlist[0] + "[title]"
# the count of pubmed ids we want to get
MAX_COUNT = 10
# get the idlist of the search
handle = Entrez.esearch(db='pubmed', retmax=MAX_COUNT, term=TERM)
result = Entrez.read(handle)
idlist = result['IdList']
# get the information of idlist
h = Entrez.efetch(db='pubmed', id=idlist, rettype='medline', retmode='text')
records = Medline.parse(h)

In [34]:
# filter the pubmed list of information to find the match
# The criteria is yet to be refined 
for r in records:
    print(r['TI'])
    print(r['PMID'])
print(TERM)

Importance of placing Gore-Tex in the subperiosteal plane for augmentation rhinoplasty.
22801183
Gore-Tex nasal augmentation: a 26-year perspective.
21422447
Are polytetrafluoroethylene (Gore-Tex) implants an alternative material for nasal dorsal augmentation in Asians?
21119414
Histological study of expanded polytetrafluoroethylene (Gore-Tex) implanted in the human nose.
19146003
The use of expanded polytetrafluoroethylene (Gore-Tex) in rhinoplasty.
17549553
Augmentation of the nasal dorsum using Gore-Tex: intermediate results of a retrospective analysis of experience in 66 patients.
11437844
Nasal augmentation using Gore-Tex. A 10-year experience.
10937089
Gore-tex augmentation grafting in rhinoplasty--is it safe?
9857319
Gore-Tex nasal augmentation.
8532794
Gore-Tex for nasal augmentation: a recent series and a review of the literature.
8534022
Nasal Augmentation Using Gore-Tex[title]


## Next Steps
1. I assume that the first several lines (default to 2 lines) is the title lines especially for the first line. The criteria for the title lines should be refined. For example, use the other boundary lines. 
2. Technique to be used: Collect several papers without doi in the text. Manually extract the title lines and store it in a variable. Use Machine Learning to set the boundary for me.

In [6]:
parsed = parser.from_file('10.1097@PRS.0000000000004961.pdf'
                         )

In [9]:
parsed['content'].strip().split('\n\n')

['D\now',
 'nloaded\nfrom',
 'http://journals.lw\nw\n.com',
 '/plasreconsurg\nby',
 'BhD\nM\nf5ePH',
 'KbH\n4TTIm',
 'qenVJjvVVJU\nIl4C',
 'sU\n4Erw',
 'LW\nFlzXzxFR',
 'k29C\nzC',
 '6Ej4rBA0by\non',
 '10/25/2018',
 'Downloadedfromhttp://journals.lww.com/plasreconsurgbyBhDMf5ePHKbH4TTImqenVJjvVVJUIl4CsU4ErwLWFlzXzxFRk29CzC6Ej4rBA0byon10/25/2018',
 'Copyright © 2018 American Society of Plastic Surgeons. Unauthorized reproduction of this article is prohibited. ',
 'www.PRSJournal.com742e',
 'Since its first description by Gonzalez-Ulloa in 1960, augmentation-mastopexy has con-tinued to evolve into its many permutations \nand become a close ally of the plastic surgeon.1,2 \nAs litigation has become commonplace within the \nmedical community, plastic surgery has also felt \nits reach, and augmentation-mastopexy remains \none of most litigated procedures in the cosmetic \npopulation.3 Perhaps the root of dissatisfaction \nand pursuit of litigation is the perpetual chal-\nlenge augmentation-